In [52]:

# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import jupyter_utils
importlib.reload(jupyter_utils)

<module 'jupyter_utils' from '/home/pa4-CarterKekoa/jupyter_utils.py'>

# PA4 Simple Classifiers
## Part 2: Auto Dataset Classification  
Write a Jupyter Notebook (pa4.ipynb) that uses your mysklearn package to build simple classifiers for the "pre-processed" automobile dataset (auto-data-removed-NA.txt) you created for PA2. In the Notebook, describe the steps, log any assumptions and/or issues you had in doing the steps, and provide insights on the step results. All re-usable utility functions should be separate from your Notebook in an appropriate module.

### Step 1 Train/Test Sets: Random Instances and Linear Regression  
Create a classifier that predicts mpg values using (least squares) linear regression based on vehicle weight. Your classifier should take one or more instances, compute the predicted MPG values, and then map these to the DOE classification/ranking (given in PA3) for each corresponding instance. Test your classifier by selecting 5 random instances from the dataset, predict their corresponding mpg ranking, and then show their actual mpg ranking as follows:

* Changes:
    * I had an issue with using fit() on two different lists, one has nested list values and one was a list of values. I had to add to the fit() function to account for this issue. 
* Data:
    * The predictions were correct 4/5 times consistantly which is pretty good! However the cars that there were less instances of were not usually guessed correctly.



In [56]:
import os
import random

# Get the file data
fname = os.path.join("input_data", "auto-data-removed-NA.txt")
auto_data = MyPyTable().load_from_file(fname)

# Grab the car weights column and store in a list
weight = auto_data.get_column('weight')
weight = [[val] for val in weight]
# split the data
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(weight, auto_data.get_column('mpg'), shuffle=True)

# get the regression line information
myline = MySimpleLinearRegressor()
myline.fit(x_train_split,y_train_split)

# randomize the data for 5 rows
rand_rows = jupyter_utils.get_rand_rows(auto_data, 5)
print("===========================================")
print("STEP 1: Linear Regression MPG Classifier")
print("===========================================")

# for row in the random row list
for row in rand_rows:
    print('instance:', row)
    print('class:', jupyter_utils.get_rating(myline.predict([[row[4]]])[0]), 'actual: ',juputils.get_rating(row[0]))

STEP 1: Linear Regression MPG Classifier
instance: [20.0, 8.0, 262.0, 110.0, 3221.0, 13.5, 75.0, 1.0, 'chevrolet monza 2+2', 3953.0]
class: 5 actual:  5
instance: [14.0, 8.0, 454.0, 220.0, 4354.0, 9.0, 70.0, 1.0, 'chevrolet impala', 3132.0]
class: 1 actual:  2
instance: [22.0, 6.0, 250.0, 105.0, 3353.0, 14.5, 76.0, 1.0, 'chevrolet nova', 3413.0]
class: 4 actual:  5
instance: [32.8, 4.0, 78.0, 52.0, 1985.0, 19.4, 78.0, 3.0, 'mazda glc deluxe', 3145.0]
class: 7 actual:  8
instance: [20.5, 6.0, 231.0, 105.0, 3425.0, 16.9, 77.0, 1.0, 'buick skylark', 3865.0]
class: 4 actual:  5


### Step 2 Train/Test Sets: Random Instances and kNN  
Create a nearest neighbor classifier for mpg that uses the number of cylinders, weight, and acceleration attributes to predict mpg for k = 5. Be sure to normalize the three attribute values and also use the Euclidean distance metric. Similar to Step 1, test your classifier by selecting 5 random instances from the dataset, predict their corresponding mpg ranking, and then show their actual mpg ranking:
* Changes:
    * A scale function needed to be made to scale the data to be between [0, 1]


In [57]:
print("===========================================")
print("STEP 2: k=5 Nearest Neighbor MPG Classifier")
print("===========================================")

mknc = MyKNeighborsClassifier(5) # set the amount of neighbors to 5

# Get the 3 column data sets
weight = auto_data.get_column('weight') 
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

# store each columns data in x_vals and scale the data
x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, blank = myutils.scale(x_vals, [[]])

# train and split the test list
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(scaled_x, auto_data.get_column('mpg'), shuffle=True)

mknc.fit(x_train_split, auto_data.get_column('mpg')) # fit the data
rand_rows = jupyter_utils.get_rand_rows(auto_data, 5) # randomize 5 rows

# for each row in the random list of 5
for row in rand_rows:
    print('instance:', row)
    scaled_vals, scaled_test = myutils.scale(x_vals, [[row[4],row[1],row[5]]])
    print('class:', jupyter_utils.get_rating(mknc.predict(scaled_test)[0]), 'actual: ', juputils.get_rating(row[0]))

STEP 2: k=5 Nearest Neighbor MPG Classifier
instance: [16.0, 6.0, 258.0, 110.0, 3632.0, 18.0, 74.0, 1.0, 'amc matador', 3699.0]
class: 5 actual:  3
instance: [21.0, 4.0, 140.0, 72.0, 2401.0, 19.5, 73.0, 1.0, 'chevrolet vega', 2192.0]
class: 6 actual:  5
instance: [19.0, 3.0, 70.0, 97.0, 2330.0, 13.5, 72.0, 3.0, 'mazda rx2 coupe', 2290.0]
class: 1 actual:  4
instance: [22.0, 6.0, 250.0, 105.0, 3353.0, 14.5, 76.0, 1.0, 'chevrolet nova', 3413.0]
class: 4 actual:  5
instance: [27.2, 4.0, 141.0, 71.0, 3190.0, 24.8, 79.0, 2.0, 'peugeot 504', 8040.0]
class: 1 actual:  7


### Step 3 Train/Test Sets: Random Sub-sampling
Compute the predictive accuracy and error rate of the two classifiers using random sub-sampling with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):
* This one was a bit difficult but didnt require me to create any new funcitons to pull it off.
* Data:
    * The accuracy seemed to run a bit low which means that the algorithm was not regularly correct in its predictions. This also means the error rate was high. 


In [58]:
print("===========================================")
print("STEP 3: Predictive Accuracy")
print("===========================================")
print("Random Subsample (k=10, 2:1 Train/Test)")

# grab the auto data
auto_data = MyPyTable().load_from_file(fname)

# get the column data 
weight = auto_data.get_column('weight')
weight = [[val] for val in weight]

# split the data
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(weight, auto_data.get_column('mpg'), shuffle=True)

# train the regression line
myline = MySimpleLinearRegressor()
myline.fit(x_train_split,y_train_split)

# predict using the regression line
predicted = myline.predict(x_test_split)
predicted = [jupyter_utils.get_rating(val) for val in predicted]
expected = [jupyter_utils.get_rating(val) for val in y_test_split]  # store the expected values

correct = 0
# for each index value in the predicted list
for i in range(len(predicted)):
    if predicted[i] == expected[i]:
        correct += 1
print('Linear Regression: accuracy = ', correct/len(predicted), 'error rate =', 1-correct/len(predicted))

# grab the colums individual
weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

# store the colum values in the x_vals
x_vals = [[weight[i], cylinders[i], acceleration[i]] for i in range(len(weight))]
scaled_x, scaled_test = myutils.scale(x_vals, [[]]) # scale them

# split the data
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(scaled_x, auto_data.get_column('mpg'), shuffle=True)

mknc = MyKNeighborsClassifier(10) # set the amount of neighbors to 10
# fit the data
mknc.fit(x_train_split, y_train_split)

# predict using the regression line
predicted = mknc.predict(x_test_split)
predicted = [jupyter_utils.get_rating(val) for val in predicted]
expected = [jupyter_utils.get_rating(val) for val in y_test_split]

correct = 0
# for each index value in the predicted list
for i in range(len(predicted)):
    if predicted[i] == expected[i]:
        correct += 1
print('K Nearest Neighbors: accuracy = ', correct/len(predicted), 'error rate =', 1-correct/len(predicted))

STEP 3: Predictive Accuracy
Random Subsample (k=10, 2:1 Train/Test)
Linear Regression: accuracy =  0.38372093023255816 error rate = 0.6162790697674418
K Nearest Neighbors: accuracy =  0.2441860465116279 error rate = 0.7558139534883721


### Step 4 Train/Test Sets: Cross Validation
Compute the predictive accuracy and error rate of the two classifiers using separate training and test sets. You should use mpg rankings for both k-fold cross validation and stratified k-fold cross validation with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):
* Corrections:
    * I needed to make a function that counted the predictions that were correct. 
* Data:
    * The predictions for the cross validation were actually pretty high menaing it was able to predict rather well.
    * However the stratified k fold was not regularly very accurate.

In [60]:
# Reload the libraries
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.myevaluation)
importlib.reload(mysklearn.myutils)

X_train = []
y_train = []

print("===========================================")
print("STEP 4: Predictive Accuracy")
print("===========================================")
print("10-Fold Cross Validation")

# get the column data 
weight = [[val] for val in auto_data.get_column('weight')]
train_folds, test_folds = myevaluation.kfold_cross_validation(weight, 10)

# grab the mpg column
mpg = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]

# fold the column data
X_train, y_train, X_test, y_test = myutils.get_from_folds(weight, mpg, train_folds, test_folds)

# fit the line to a regression
myline = MySimpleLinearRegressor()
myline.fit(X_train,y_train)

# predict  the data
prediction = [round(val) for val in myline.predict(X_test)]
actual_linear = y_test

# count the amount of correct predictions
count = jupyter_utils.correct_count(prediction, y_test)
print('Linear Regression: accuracy =', count/len(prediction), 'error rate =', 1 - count/len(prediction))

# grab the colums individual
weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

# store the colum values in the x_vals
x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, _ = myutils.scale(x_vals, [[]])

# make the folds
train_folds, test_folds = myevaluation.kfold_cross_validation(scaled_x, 10)
y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]

# get the train and test lists
X_train, y_train, X_test, y_test = myutils.get_from_folds(scaled_x, y, train_folds, test_folds)

mknc = MyKNeighborsClassifier(10) # set the amount of neighbors to 10
mknc.fit(X_train,y_train)

# predict the neighbors in test
predicted_neighbors = mknc.predict(X_test)
actual_neighbors = y_test
count = jupyter_utils.correct_count(predicted_neighbors, y_test) # count the amount of correct neighbors predicted

print('K Nearest Neighbors: accuracy =', count/len(predicted_neighbors), 'error rate =', 1 - count/len(predicted_neighbors))

X_train = []
y_train = []

# grab the weigbt column
weight = [[val] for val in auto_data.get_column('weight')]
y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(weight, y, 10)

# Get the ratings for each of the mpg values
ratings = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]

# grab the folds
X_train, y_train, X_test, y_test = myutils.get_from_folds(weight, ratings, train_folds, test_folds)

# fit the Regression line
myline = MySimpleLinearRegressor()
myline.fit(X_train,y_train)

# make the predictions
prediction_stratified = [round(val) for val in myline.predict(X_test)]
actual_linear_stratified = y_test

# count the amount of correct predictions
count = jupyter_utils.correct_count(predicted_linear_stratified, y_test)

print('\nStratified 10-Fold Cross Validation\nLinear Regression: accuracy =', count/len(predicted_linear_stratified), 'error rate =', 1 - count/len(predicted_linear_stratified))

# grab the colums individual
weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

# store the colum values in the x_vals
x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, blank = myutils.scale(x_vals, [[]])

# Get the ratings for each of the mpg values
rating = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(scaled_x, rating, 10)

# Grabt he folds
X_train, y_train, X_test, y_test = myutils.get_from_folds(scaled_x, rating, train_folds, test_folds)

# set the amount of neighbors to 10
mknc = MyKNeighborsClassifier(10)
mknc.fit(X_train,y_train)

# predict the neighbors
predicted_neighbors_stratified = mknc.predict(X_test)
actual_neighbors_stratified = y_test

# count the amount of correct predictions
count = jupyter_utils.correct_count(predicted_neighbors_stratified, y_test)

print('K Nearest Neighbors: accuracy =', count/len(predicted_neighbors_stratified), 'error rate =', 1 - count/len(predicted_neighbors_stratified))

STEP 4: Predictive Accuracy
10-Fold Cross Validation
Linear Regression: accuracy = 0.8185328185328186 error rate = 0.18146718146718144
K Nearest Neighbors: accuracy = 0.528957528957529 error rate = 0.471042471042471

Stratified 10-Fold Cross Validation
Linear Regression: accuracy = 0.3166023166023166 error rate = 0.6833976833976834
K Nearest Neighbors: accuracy = 0.14671814671814673 error rate = 0.8532818532818532


### Step 5 Confusion Matrices  
Create confusion matrices for each classifier based on the stratified 10-fold cross validation results. You can use the tabulate package to display your confusion matrices (it is also okay to format the table manually). Here is an example:
* Data:
    * It seems the matrix was not able to guess the correct label very often.



In [61]:
importlib.reload(jupyter_utils)
print('\n===========================================\n\
STEP 5: Confusion Matrices\n\
===========================================\n')
matrix = myevaluation.confusion_matrix(actual_linear_stratified, predicted_linear_stratified, [0,1,2,3,4,5,6,7,8,9,10])
table_header = ['MPG', 1,2, 3 ,4, 5, 6, 7, 8, 9, 10, 'Total', 'Recognition (%)']
jupyter_utils.add_conf_stats(matrix)

print('Linear Regression (Stratified 10-fold Cross Validation Results)')
jupyter_utils.print_tabulate(matrix, table_header)

matrix = myevaluation.confusion_matrix(actual_neighbors_stratified, predicted_neighbors_stratified, [0,1,2,3,4,5,6,7,8,9,10])
jupyter_utils.add_conf_stats(matrix)

print('\nK Neighbors (Stratified 10-fold Cross Validation Results)')
jupyter_utils.print_tabulate(matrix, table_header)


STEP 5: Confusion Matrices

Linear Regression (Stratified 10-fold Cross Validation Results)
=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
  MPG    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
    1    8   10    8    0    1    0    0    0    0     0       28              28.57
    2    2    9    4    0    1    0    0    0    0     0       18              50
    3    1   14   12    8    1    0    0    0    0     0       39              30.77
    4    0    1   11   25   11    2    1    0    0     0       55              45.45
    5    0    0    1   12   19   15    0    0    0     0       52              36.54
    6    0    0    0    1    6   21    6    0    0     0       40              52.5
    7    0    0    0    1    0    8   18    0    0     0       34              52.94
    8    0    0    0    0    0    1   16    1    0     0     